### Loading transcript

In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import os
import shutil
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter, WebVTTFormatter, SRTFormatter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.vectorstores import InMemoryVectorStore
import chromadb
from langchain_core.tools import tool
from langchain_core.documents import Document
from chromadb.config import Settings
import streamlit as st
import tempfile

In [2]:
def get_youtube_transcription(video_url, language_code=['pt']):
  # Extrair o ID do vídeo a partir do URL
  video_id = video_url.split("v=")[-1]
  if "&" in video_id:
      video_id = video_id.split("&")[0]
  # Obter a transcrição no idioma especificado
  transcript = YouTubeTranscriptApi.get_transcript(video_id,languages=language_code)
  return transcript


def format_transcript(transcript):
  tempo = 0
  formated_list = []
  frase = ""
  for t in transcript:
    tempo += t['duration']
    frase += t['text']
    if tempo >= 300:
      formated_list.append({"text":frase,"start":tempo})
      tempo = 0
      frase = ""
    formated_list.append({"text":frase,"start":round(tempo)})
  return formated_list

def format_doc(docs,link):
  formated_docs = []
  for doc in docs:
    link_tempo = link + "&t=" + str(doc["start"]) + "s"
    document = Document(
            page_content=doc['text'],
            metadata={"source": link_tempo,"time":doc["start"]}
        )
    formated_docs.append(document)
  return formated_docs

In [3]:
link = "https://www.youtube.com/watch?v=jrRow5ao6So"

In [5]:
doc_splits = [Document(metadata={'source': 'https://www.youtube.com/watch?v=jrRow5ao6So&t=303s', 'time': 303}, page_content='É surreal o dólar segue em disparadaquando eu comecei a gravar esse vídeo eutive que recomeçar o câmbio tava em 6:20E aí foi para 6:21 6:22 6:23 6:24 bateur$ 66,25 tô gravando aqui na quarta-feira 18de dezembro às 16:20 da tarde e o câmbiosegue subindo está fora de controleAssim como as taxas de juros que tambémestão em alta porque não há nenhum sinalde reversão de curso governo segue serecusando a ouvir o entender aspreocupações genuínas de muita gente coma situação fiscal com o descontrole dascontas públicas Parece que ainda nãocaiu a ficha em Brasília sobre agravidade do que está acontecendo e essenem er o assunto do vídeo mas como ocâmbio começou a subir sem parar eu tiveque abordar o assunto mas na verdade oque eu queria agora era responder duasperguntas que surgiram no vídeo de ontemRecomendo vocês assistirem Quem aindanão o fez vou colocar o link aqui emcima a primeira que era sobre adiferença entre o dólar em quase seis naépoca de bolsonaro e Paulo Guedes e odólar agora em 6:25 com o Lula e Haddade a segunda pergunta era Será que oBrasil pode se tornar a Turquia porquelá a lira turca também se desvalorizou emuito nos últimos anos então vouresponder essas duas perguntas Primeirocomeçando então pela comparação comPaulo Guedes e adad o que que nós temosassim que a gente poderia sumarizar adiferença de agora ora com na época deGuedes Paulo Guedes tentou uma políticade controle e aperto fiscal e frouxidãomonetária então era o fiscal controladoapertado e o juro mais frouxo agora é ocontrário temos um juro lá em cima umaperto monetário e um fiscal muitofrouxo claro que quando veio a pandemiaisso desarranjo e bastante mas valecolocar em perspectiva e entender tambéma diferença nas magnitudes dosmovimentos na taxa de de câmbio entãocolocando aqui primeiro agora em 2024Quanto que o câmbio está sedesvalorizando quase29 apenas em 2024 que está subindo ocâmbio mas quando a gente compara lá com2020 Vamos dar um zoom out aqui nessegráfico Vejam Só Janeiro de 2020 ocâmbio estava em4,02 em maio 2020 O câmbio subiu quase47% chegou em 5,90 chegou até bater nointra diário 5,94 mas enfim aqui quase'),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=jrRow5ao6So&t=301s', 'time': 301}, page_content='5,90 em 5 meses o câmbio chegou a subirquase 50% não é o que temos agora sefizéssemos essa mesa comparação parachegar em ó vamos lá de até da da mínimado governo atual que foi em julho do anodo ano passado até agora subiu33% mas em mais de 1 ano e 2 3 mesesentão tem uma diferença grande comaconteceu lá com o Paulo Guedes masclaro que a pandemia foi o grandeagravante só que havia o problema e queeu apontei muito antes da pandemia que ogoverno a o ministro da Fazenda PauloGuedes junto com o Roberto Campos Netoeles estavam perseguindo uma políticamonetária muito frouxa colocando o jurolá embaixo e na minha avaliação longe doque seria os fundamentos da economiabrasileira não era compatível com anossa economia um país emergente com osriscos que nós tínhamos ainda naquelemomentomas essa foi a política praticada Eu atévou colocar aqui na tela a taxa de jurosvamos lá aqui ó Brasília interest ratepra gente comparar com o que aconteceunaquele momento cá temos então que quetínhamos lá em 2019 até vou voltar umpouquinho ó 2019 a taxa de juros Começoua cair cair e chegou no finalzinho de2019 em 4 e 5% foi a taxa básica dejuros e o câo já tinha começado a subirentão Saiu quando o bolsonaro assumiu ogoverno aqui ó bolsonaro assumiu ogoverno a taxa de câmbio estava ainda em3,75 e acabou2019 com a taxa de câmbio já em quatroentão ali começou um uma taxa de jurofora do lugar porém Claro a boa notíciaé que o fiscal estava controlado masquando chegou a pandemia e veio todos osproblemas os temores no mundo inteiro ocâmbio disparou chegou nesses quase seise e o banco central começou uma políticade mais frouxidão ainda baixando aindamais o juro chegando em 2% o que eucritiquei enormemente em vários vídeosaqui no canal e além agora do juro forado lugar muito frouxo o fiscal tambémfoi pro espaço com todas as exigênciasda pandemia auxílio emergencial o rombofiscal foi gigantesco histórico como jámostrei em vários vídeos Talvez ospróximos meses o do atual governo possaaté superar até mesmo da pandemia pelomenos o déficit nominal mas naquele'),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=jrRow5ao6So&t=303s', 'time': 303}, page_content='momento era o déficit primário que foirealmente e enorme por conta daquelaconjuntura mas à medida que a gente foisaindo da pandemia e que a economia foise recuperando e o Paulo Guedes voltoucom a sua prioridade de controle fiscalde realmente colocar a casa em ordem e obanco central também se deu conta queera um erro colocar a taxa de juros em2% abandonou Esse regime monetário ePass a subir e muito a taxa de juros echegamos até ali em 2022 Abril já em 11e 75% e o Cambo já tinha cedido bastantechegou até nas mínimas ali de 4,57 noano de 2022 e depois juro seguiu subindoe aí o restante da história do 22 e 23 Agente já sabe mas então essa era agrande diferença daquele momento em quehavia uma preocupação genuína umaintenção de buscar controle fiscal e nomínimo um déficit primário erado o queestava fora do lugar era o juro muitofrouxo e quando veio a pandemia estouroumais o fiscal juro frouxo tudo issoagravou o problema e o câmbio decoloudepois que o fiscal foi arrumado tivemossuperavit em2021 e 22 também superavit primário e ojuro também foi pro lugar maiscompatível com a nossa economia tambémacho que quase 14 ali que foi o 3,75também acho que foi exagerado Mas enfimo juro não estava naquele patamar tãobaixo que não não era correto erainadequado e levou o câmbio a ficar forado lugar e descontrolado mas veio entãoo Lula e Haddad com o discurso o planode governo a campanha de lula que nãoera de controle fiscal de verdade eleinclusive em algumas declarações falandoque gasto era vida aquilo que eu jácansei de falar várias vezes mas poralguns meses o mercado deu o benefícioda dúvida ainda estava esperançoso quepoderia haver o um controle fiscalGenuíno Porém quando veio quando vieramas primeiras notícias do arcabouo fiscalaí ficou Claro que não havia seriedadenão havia compromisso de fato com ocontrole das contas públicas a situaçãoera bem mais séria do que se imaginava ea medida que o tempo foi passando issosó piorou nem o arcabouço que já erafrouxo e muito flexível leniente nem oarcabouo foi está sendo cumprido e oresultado agora então essa desconfiançacompleta essa falta de credibilidade emque temos um fiscal muito frouxo e mesmo'),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=jrRow5ao6So&t=301s', 'time': 301}, page_content='com o monetário com o juro muito alto Éum aperto monetário está sendoinsuficiente para resolver a questão Eaí que eu digo que hoje parece ser maiscomplexo resolver o problema do câmbionas alturas porque na época do PauloGuedes era relativamente mais simplescara o principal problema era o jurofora do lugar Banco Central Para debrincar de país de primeiro mundo começaa subir juros e as coisas vão voltar aosseu patamar aos seus patamares normais efoi que aconteceu câmbio voltou a cairjuro subiu câmbio caiu e o fiscal tambémvoltou a entrar em ordem 2021 e 22 agoraé mais difícil porque o juro já tá nasalturas segue subindo e o problema estáno fiscal e não há nenhuma perspectivanão há intenção genuína não hácompromisso não há plano congressoleniente também se fazendo de louco quenão tá entendendo o que está acontecendoe a ação é essa câmbio disparada nessemomento aqui R6,24 sei lá Quanto que vai fechar hojemas é é realmente um desastre pro paísisso logo vai tocar na inflação e apopulação que vai empobrecer e essaespiral negativa que ninguém consegueinterromper é é complicado agora asegunda pergunta sobre a comparaçãoentre o real brasileiro e a lira turcaQual é a diferença do Lula e do erdoganque é o premier da Turquia vale a penatambém responder mas eu preciso lembrarvocês do nosso aplicativo follow themoney Vou colocar aqui o código kerk emcima também porque nós fomos pra Turquiano ano passado para estar lá em loco edocumentar a realidade do país conversarcom as pessoas nas ruas entender o queestava acontecendo e esse foi um dosprincipais documentários que a genteproduziu pra Série nas fronteiras dodinheiro que está no aplicativo followda Money em que a gente explica todas asrazões por trás do movimento da Liraturca que também Saiu em disparada Masvamos primeiro colocar aqui a comparaçãono momento Lira turca e real brasileiroem 2024 a nossa ta de câmbio quase 29por de alta a lira quase 19% conseguimosser uma moeda pior do que a lira turcaem 2024 mas claro quando a gente dá umzo out e entende a diferença do queaconteceu lá e o que está acontecendo noBrasil é quase Incomparável Vamos voltarlá para')
 ]

In [17]:
def vector_store():
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=OPEN_API_KEY)
  vectorstore = InMemoryVectorStore(embeddings)
  return vectorstore

In [21]:
def load_doc_to_db(doc_splits):
  print(doc_splits)
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large",api_key=OPEN_API_KEY)
  db = vector_store()
  # Add to vectorDB
  db.from_documents(embedding=embeddings,documents=doc_splits)
  print("loaded")
  return db

In [22]:
db = load_doc_to_db(doc_splits)

[Document(metadata={'source': 'https://www.youtube.com/watch?v=jrRow5ao6So&t=303s', 'time': 303}, page_content='É surreal o dólar segue em disparadaquando eu comecei a gravar esse vídeo eutive que recomeçar o câmbio tava em 6:20E aí foi para 6:21 6:22 6:23 6:24 bateur$ 66,25 tô gravando aqui na quarta-feira 18de dezembro às 16:20 da tarde e o câmbiosegue subindo está fora de controleAssim como as taxas de juros que tambémestão em alta porque não há nenhum sinalde reversão de curso governo segue serecusando a ouvir o entender aspreocupações genuínas de muita gente coma situação fiscal com o descontrole dascontas públicas Parece que ainda nãocaiu a ficha em Brasília sobre agravidade do que está acontecendo e essenem er o assunto do vídeo mas como ocâmbio começou a subir sem parar eu tiveque abordar o assunto mas na verdade oque eu queria agora era responder duasperguntas que surgiram no vídeo de ontemRecomendo vocês assistirem Quem aindanão o fez vou colocar o link aqui emcima a primeira

In [24]:
db.similarity_search(query="taxa")

[]